In [1]:
import os
import glob
import time
import cv2
import numpy as np
import torch
from torch.autograd import Variable
from utils import *  # Đảm bảo file utils.py nằm cùng thư mục hoặc trong PYTHONPATH
from networks import *  # Tương tự cho networks.py


In [6]:
# Cấu hình đường dẫn và thông số
logdir = r"D:\DSP_Project\derain_PReNet\PReNet\logs\finetuning\net_epoch49_med.pth"
data_path = r"C:\Users\ADMIN\Desktop\test_real\rainy\heavy"
save_path = r"C:\Users\ADMIN\Desktop\test_real\results"
use_GPU = True
gpu_id = "0"
recurrent_iter = 6

# Thiết lập GPU
if use_GPU:
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu_id

In [7]:
os.makedirs(save_path, exist_ok=True)

In [8]:
# Khởi tạo mô hình
print('Loading model...\n')
model = PReNet(recurrent_iter, use_GPU)
print_network(model)

if use_GPU:
    model = model.cuda()

# Load trọng số
model.load_state_dict(torch.load(logdir))
model.eval()

Loading model...

PReNet(
  (conv0): Sequential(
    (0): Conv2d(6, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (res_conv1): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
  (res_conv2): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
  (res_conv3): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
  (res_conv4): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
  (re

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_5620\2309444080.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(logdir))


PReNet(
  (conv0): Sequential(
    (0): Conv2d(6, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (res_conv1): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
  (res_conv2): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
  (res_conv3): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
  (res_conv4): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
  (res_conv5): Sequenti

In [9]:
time_test = 0
count = 0

for img_name in os.listdir(data_path):
    if is_image(img_name):
        img_path = os.path.join(data_path, img_name)

        # Đọc và chuẩn hóa ảnh
        y = cv2.imread(img_path)
        b, g, r = cv2.split(y)
        y = cv2.merge([r, g, b])  # Chuyển sang RGB

        y = normalize(np.float32(y))
        y = np.expand_dims(y.transpose(2, 0, 1), 0)
        y = Variable(torch.Tensor(y))

        if use_GPU:
            y = y.cuda()

        with torch.no_grad():
            if use_GPU:
                torch.cuda.synchronize()
            start_time = time.time()

            out, _ = model(y)
            out = torch.clamp(out, 0., 1.)

            if use_GPU:
                torch.cuda.synchronize()
            end_time = time.time()

            dur_time = end_time - start_time
            time_test += dur_time
            print(img_name, ':', dur_time)

        # Chuyển tensor về ảnh
        out_np = out.data.cpu().numpy().squeeze() if use_GPU else out.data.numpy().squeeze()
        save_out = np.uint8(255 * out_np).transpose(1, 2, 0)
        b, g, r = cv2.split(save_out)
        save_out = cv2.merge([r, g, b])  # Trở lại BGR để lưu bằng OpenCV

        cv2.imwrite(os.path.join(save_path, img_name), save_out)
        count += 1

print('Avg. time:', time_test / count)


hv1.jpg : 0.4136533737182617
hv2.jpg : 0.3827829360961914
Avg. time: 0.39821815490722656
